In [8]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [9]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [10]:
system_message = 'You are a helpful assistant'

In [18]:
def message_llama(prompt):
    message= [
        {"role": "system", "content": system_message},
        {"role": "user", "content":prompt}
    ]
    completion = openai.chat.completions.create(
        model="llama3.2", messages=message
    )
    return completion.choices[0].message.content

In [20]:
message_llama("what can you do?")

"I can be used in a variety of ways, from helping you plan a vacation to creating art. I'm here to assist you in finding the help or information you need. My strengths include answering questions, generating text and images and even just chatting with you."

In [21]:
def shout(text):
    return text.upper()

In [22]:
shout("hello")

'HELLO'

In [29]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", allow_flagging="never").launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://c30122f457a696c7c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [30]:
gr.Interface(fn=message_llama, inputs='textbox', outputs='textbox', allow_flagging='never').launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [32]:
system_message = 'you are a helpful assistant that responds in markdown'

view = gr.Interface(fn=message_llama,
                    inputs=[gr.Textbox(label="Your Message")],
                    outputs=[gr.Markdown(label="Response")],
                    allow_flagging='never'

                    )
view.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
